# Extracting, transforming and selecting features

This section covers algorithms for working with features, roughly divided into these groups:

* (partially) Extraction: Extracting features from “raw” data
* (partially) Transformation: Scaling, converting, or modifying features
* (skip)Selection: Selecting a subset from a larger set of features
* (skip)Locality Sensitive Hashing (LSH): This class of algorithms combines aspects of feature transformation with other algorithms. 

## Feature Extractors
### TF-IDF (Term frequency-inverse document frequency (TF-IDF)
* a feature vectorization method widely used in text mining to reflect the importance of a term to a document in the corpus. 
* Terminologies 
   + t: a term or a phrase
   + d: a document 
   + D: the corpus by D
* Term frequency TF(t,d) = the number of times that term t appears in document d
   + (참고) https://en.wikipedia.org/wiki/Tf%E2%80%93idf
   + 다른 정의
      - the raw count itself: tf(t,d) = ft,d
      - Boolean "frequencies": tf(t,d) = 1 if t occurs in d and 0 otherwise;
      - term frequency adjusted for document length: tf(t,d) = ft,d ÷ (number of words in d)
      - logarithmically scaled frequency: tf(t,d) = log (1 + ft,d);
      - augmented frequency, to prevent a bias towards longer documents, e.g. raw frequency divided by the raw frequency of the most frequently occurring term in the document:  
* Document frequency DF(t,D): the number of documents that contains term t. 
* IDF(t,D) = log ( (|D| +1) / (DF(t,D) +1) ) 
   + |D| the total number of documents in the corpus.
   + DF(t,D)의 반비례(역수)를 쓰는 이유: 너무 빈번하게 모든 문서에 등장하는 단어(a, the, is 등)을 배제하기 위함 
   + +1의 의미: 분모가 0가 되는 것을 방지하기 위함. 

* TFIDF(t,d,D)=TF(t,d)⋅IDF(t,D) 

#### TF: Both HashingTF and CountVectorizer can be used to generate the term frequency vectors.
* HashingTF is a Transformer which takes sets of terms and converts those sets into fixed-length feature vectors. 
* CountVectorizer converts text documents to vectors of term counts. Refer to CountVectorizer for more details.

#### IDF
* IDF is an Estimator which is fit on a dataset and produces an IDFModel. 
* The IDFModel takes feature vectors (generally created from HashingTF or CountVectorizer) and scales each feature. Intuitively, it down-weights features which appear frequently in a corpus.

In [0]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)
wordsData.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish Java could...|[i, wish, java, c...|
|  1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [0]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
featurizedData.show()
# alternatively, CountVectorizer can also be used to get term frequency vectors


+-----+--------------------+--------------------+--------------------+
|label|            sentence|               words|         rawFeatures|
+-----+--------------------+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|(20,[6,8,13,16],[...|
|  0.0|I wish Java could...|[i, wish, java, c...|(20,[0,2,7,13,15,...|
|  1.0|Logistic regressi...|[logistic, regres...|(20,[3,4,6,11,19]...|
+-----+--------------------+--------------------+--------------------+



In [0]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20,[6,8,13,16],[...|
|  0.0|(20,[0,2,7,13,15,...|
|  1.0|(20,[3,4,6,11,19]...|
+-----+--------------------+

